# Data processing for the SIM model

Initial data handling for preparing material to be used in [SpatInteModel.ipynb](SpatInteModel.ipynb).

## Initialisation

Uploading libraries and data

In [ ]:
# Loading libraries
import os
import pandas as pd
import geopandas as gpd
from pathlib import Path
import matplotlib.pyplot as plt

# Dealing with the data
## Paths definition 
base_dir = str(Path(os.getcwd()).parent)  # Get main directory (should be parent to this file)
data_dir = os.path.join(base_dir,
                        "data",
                        "national_data") 
results_dir = os.path.join(base_dir,
                           "output")

if os.path.basename(base_dir) != "FutureHighStreets": #base_dir.split("/")[-1] != "FutureHighStreet":
    raise Exception(f"The base directory should point to the main project directory,"
                    f"but it points to {base_dir}")
    
## Variables definition
origin_data_folder = "LSOA"
origin_data_filename = "Lower_Layer_Super_Output_Areas_(December_2011)_Population_Weighted_Centroids"  # change depending on costum edits
origin_shp_path = os.path.join(data_dir,
                                     origin_data_folder,
                                     origin_data_filename) + ".shp"
destination_data_folder = "Retail_Centres"
destination_data_filename = "Retail_Centres_UK" # change depending on costum edits
destination_gpkg_path = os.path.join(data_dir,
                         destination_data_folder,
                         destination_data_filename) + ".gpkg"

## Loading data
# origin_table = pd.read_csv(os.path.join(data_dir,
#                                       origin_data_folder,
#                                       origin_data_filename))
# origin_csv.head() 
# destination_csv = pd.read_csv(os.path.jpin(data_dir,
#                                            destiantion_data_folder,
#                                            destination_data_filename))
origin_shp = gpd.read_file(origin_shp_path)
origin_shp.head()

destination_gpkg = gpd.read_file(destination_gpkg_path,
                    layer = destination_data_filename)
destination_gpkg.head()


# Actual data processing

Generating the centroids and other processed data

In [ ]:
# find the correct functions!

# Generating local data for the model

Create folders for area of interest where to run the SIM, for example LAD. This folder can be then called from within the model, that is [SpatInteModel.ipynb](SpatInteModel.ipynb)